# Drive and Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.4 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
import os
from IPython.display import display, Image
from IPython import display
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import shutil
from joblib import Parallel, delayed
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# Setup Dataset (note: we're using 10% of YOLO dataset)

In [ ]:
DATA_ROOT = "/content/drive/MyDrive/Datasets/SPARK"
SPARK_DETECT = DATA_ROOT + "/spark-2024-train-val"

YOLO = DATA_ROOT + "/spark-yolo"

TRAIN_CSV = os.path.join(SPARK_DETECT, "train.csv")
VAL_CSV   = os.path.join(SPARK_DETECT, "val.csv")

# Path for images and masks original
SPARK_IMAGES = os.path.join(SPARK_DETECT, "images")

# Path YOLO
IMG_TRAIN = os.path.join(YOLO, "images", "train")
IMG_VAL   = os.path.join(YOLO, "images", "val")
LAB_TRAIN = os.path.join(YOLO, "labels", "train")
LAB_VAL   = os.path.join(YOLO, "labels", "val")

for d in [IMG_TRAIN, IMG_VAL, LAB_TRAIN, LAB_VAL]:
    os.makedirs(d, exist_ok=True)

# Class mapping
class_to_id = {
    "VenusExpress": 0,
    "Cheops": 1,
    "LisaPathfinder": 2,
    "ObservationSat1": 3,
    "Proba2": 4,
    "Proba3": 5,
    "Proba3ocs": 6,
    "Smart1": 7,
    "Soho": 8,
    "XMM Newton": 9
}

In [ ]:
# Copy all of the dataset (YOLO dataset 10%) in Google Drive to the local content (takes 10 min)
!cp -r "/content/drive/MyDrive/Datasets/SPARK/spark-yolo-10%" /content/spark-yolo-10%

In [ ]:
# Create the data.yaml file
yaml_local = """
path: /content/spark-yolo-10%

train: images/train
val: images/val

names:
  0: VenusExpress
  1: Cheops
  2: LisaPathfinder
  3: ObservationSat1
  4: Proba2
  5: Proba3
  6: Proba3ocs
  7: Smart1
  8: Soho
  9: XMM Newton
"""
with open("/content/spark-yolo-10%/data.yaml", "w") as f:
    f.write(yaml_local)

In [ ]:
#YAML_PATH = "/content/drive/MyDrive/Datasets/SPARK/spark-yolo-10%/data.yaml"
YAML_PATH = "/content/spark-yolo-10%/data.yaml"
SAVE_PATH = "/content/drive/MyDrive/SPARK_stuff"

In [ ]:
# load the yolo11 models (nano, small, medium)
model_nano = YOLO("yolo11n.pt")
model_small = YOLO("yolo11s.pt")
model_medium = YOLO("yolo11m.pt")

In [ ]:
!nproc --all

12


In [ ]:
import numpy as np
import IPython.display as ipd

# generate a short beep (I used it to know when the training has stopped,
# beacuse you can't automatically disconnect from the runtime)
sr = 44100
freq = 880
duration = 3.2
t = np.linspace(0, duration, int(sr*duration))
wave = 0.5*np.sin(2*np.pi*freq*t)

ipd.display(ipd.Audio(wave, rate=sr, autoplay=True))

# Training experiments

In [ ]:
results1 = model_nano.train(
    data=YAML_PATH,
    epochs=100,
    imgsz=256,
    batch=256,
    workers=8,
    patience=10,
    device=0,
    amp=True,
    verbose=True,
    project=SAVE_PATH,
    name="yolo11n_256",
)

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=256, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/spark-yolo-10%/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_2563, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=

In [ ]:
results2 = model_nano.train(
    data=YAML_PATH,
    epochs=100,
    imgsz=128,
    batch=256,
    workers=8,
    patience=7,
    device=0,
    amp=True,
    verbose=True,
    project=SAVE_PATH,
    name="yolo11n_128",
)

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=256, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/spark-yolo-10%/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=128, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_128, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=7, perspective=0.

In [ ]:
results3 = model_nano.train(
    data=YAML_PATH,
    epochs=100,
    imgsz=512,
    batch=0.8,
    workers=8,
    patience=7,
    device=0,
    amp=True,
    verbose=True,
    project=SAVE_PATH,
    name="yolo11n_512_FINAL",
)

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=0.8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/spark-yolo-10%/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_512_FINAL2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=7, perspec

In [ ]:
results4 = model_nano.train(
    data=YAML_PATH,
    epochs=100,
    imgsz=640,
    batch=0.8,
    workers=8,
    patience=7,
    device=0,
    amp=True,
    verbose=True,
    project=SAVE_PATH,
    name="yolo11n_640_FINAL",
)

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=0.8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/spark-yolo-10%/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_640_FINAL, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=7, perspect

In [ ]:
# finish inference
model = YOLO("/content/drive/MyDrive/SPARK_stuff/yolo11n_640_FINAL/weights/best.pt")
results = model.val(
    data=YAML_PATH,
    imgsz=640,
    batch=256,
    device=0,
)

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
YOLO11n summary (fused): 100 layers, 2,584,102 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 222.9±257.4 MB/s, size: 82.4 KB)
val: Scanning /content/spark-yolo-10%/labels/val.cache... 2000 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 2000/2000 2.4Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 2.3s/it 18.3s
                   all       2000       2000      0.997      0.992      0.995      0.954
          VenusExpress        200        200      0.993       0.99      0.995      0.932
                Cheops        200        200      0.996      0.995      0.995      0.971
        LisaPathfinder        200        200      0.999          1      0.995      0.963
       ObservationSat1        200        200          1      0.995      0.995      0.977
                Proba2      

In [ ]:
results5 = model_nano.train(
    data=YAML_PATH,
    epochs=100,
    imgsz=1024,
    batch=100,
    workers=6,
    patience=7,
    device=0,
    amp=True,
    verbose=True,
    project=SAVE_PATH,
    name="yolo11n_1024_FINALL",
)

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=100, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/spark-yolo-10%/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_1024_FINALL, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=7, persp

In [ ]:
results6 = model_small.train(
    data=YAML_PATH,
    epochs=150,
    imgsz=640,
    batch=0.7,
    workers=7,
    patience=7,
    device=0,
    amp=True,
    verbose=True,
    project=SAVE_PATH,
    name="yolo11s_640",
)

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=0.7, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/spark-yolo-10%/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11s_640, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=7, perspective=0.

In [ ]:
results7 = model_nano.train(
    data=YAML_PATH,
    epochs=100,
    imgsz=640,
    batch=256,
    workers=7,
    patience=10,
    device=0,
    amp=True,
    verbose=True,
    project=SAVE_PATH,
    name="yolo11n_640_aug_light",

    # Geometric
    fliplr=0.5,
    flipud=0.1,
    translate=0.10,
    scale=0.1
)

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=256, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/spark-yolo-10%/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.1, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_640_aug_light3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, pe

In [ ]:
results8 = model_nano.train(
    data=YAML_PATH,
    epochs=150,
    imgsz=640,
    batch=0.4,
    workers=7,
    patience=15,
    device=0,
    amp=True,
    verbose=True,
    project=SAVE_PATH,
    name="yolo11n_640_aug_heavy",

    # geometric
    fliplr=0.5,
    flipud=0.2,
    translate=0.15,
    scale=0.2,
    degrees=10,

    auto_augment="randaugment",
)

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=0.4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/spark-yolo-10%/data.yaml, degrees=10, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.2, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_640_aug_heavy, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=15, pers

In [ ]:
results9 = model_nano.train(
    data=YAML_PATH,
    epochs=150,
    imgsz=256,
    batch=256,
    workers=8,
    patience=15,
    device=0,
    amp=True,
    verbose=True,
    project=SAVE_PATH,
    name="yolo11n_256_aug_heavy",

    # geometric
    fliplr=0.5,
    flipud=0.2,
    translate=0.15,
    scale=0.2,
    degrees=10,

    auto_augment="randaugment",
)

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=256, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/spark-yolo-10%/data.yaml, degrees=10, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.2, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_256_aug_heavy, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=15, pers

In [ ]:
results10 = model_medium.train(
    data=YAML_PATH,
    epochs=150,
    imgsz=640,   # TEST 1
    batch=0.7,
    workers=7,
    patience=7,
    device=0,
    amp=True,
    verbose=True,
    project=SAVE_PATH,
    name="yolo11m_640",
)

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=0.7, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/spark-yolo-10%/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11m_640, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=7, perspective=0.

In [ ]:
results11 = model_nano.train(
    data=YAML_PATH,
    epochs=100,
    imgsz=512,
    batch=0.8,
    workers=8,
    patience=10,
    device=0,
    amp=True,
    verbose=True,
    project=SAVE_PATH,
    name="yolo11s_512",
)

In [ ]:
results12 = model_small.train(
    data=YAML_PATH,
    epochs=250,
    imgsz=640,
    batch=0.55,
    workers=7,
    patience=30,
    device=0,
    amp=True,
    verbose=True,
    project=SAVE_PATH,
    name="yolo11s_640_aug_light",

    # Geometric
    fliplr=0.5,
    flipud=0.1,
    translate=0.10,
    scale=0.1
)

Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=0.55, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/spark-yolo-10%/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=250, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.1, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11s_640_aug_light, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=30, pe